In [5]:
import pandas as pd

In [17]:
excel_file = pd.ExcelFile('data/all_charts.xlsx')
print("Available sheets:")
for i, sheet in enumerate(excel_file.sheet_names):
    print(f"{i}: {sheet}")
print(f"\nTotal sheets: {len(excel_file.sheet_names)}")


Available sheets:
0: BMT (1,911)
1: T-Cell Therapy (85)
2: Progress Notes 1
3: Progress Notes 2
4: Consults
5: Discharge Summary
6: H&P
7: Pathology 
8: Image Report

Total sheets: 9


In [18]:
h_p_list = pd.read_excel('data/all_charts.xlsx', sheet_name='H&P', engine='openpyxl')
target_list = pd.read_excel('data/toci_ptcy_CRS_project.xlsx', engine='openpyxl')

baseline_column = ['MRN', 'BMT_date', 'Disease_x', 'age', 'Gn', 'KPS', 'CMV', 'aborh', 'Tx_Type',
                   'HLA', 'donabo', 'doncmv', 'Source_x', 'Prep', 'AB', 'gvhdpr']
research_column = ['crs_y_n', 'fever_onset_date', 'last_fever_date', 'max_temp', 'hypotension_y_n',
                   'pressor_use_num', 'hypoxia_y_n', 'high_flow_o2_y_n', 'bipap_or_intubation_y_n', 
                   'neurotox_y_n', 'toci_y_n', 'toci_start_date', 'toci_stop_date', 'total_dose_toci']

In [19]:
# filter h_p_list to keep only rows where MRN exists in target_list
h_p_filtered = h_p_list[h_p_list['MRN'].isin(target_list['MRN'])].copy()

# Group by MRN and Contact Date, find the max Line # for each combination
max_line_per_mrn_date = h_p_filtered.groupby(['MRN', 'Contact Date'])['Line #'].max().reset_index()

# Keep only MRN-Date combinations where max Line # is >= 5
valid_mrn_dates = max_line_per_mrn_date[max_line_per_mrn_date['Line #'] >= 5]

# Count how many valid dates each MRN has
mrn_date_counts = valid_mrn_dates.groupby('MRN').size().reset_index(name='date_count')

# Keep only MRNs that have multiple dates (>= 2) meeting the line number criteria
valid_mrns = mrn_date_counts[mrn_date_counts['date_count'] >= 2]['MRN']

# Filter the original data to keep only valid MRN-Date combinations
h_p_filtered = h_p_filtered.merge(
    valid_mrn_dates[['MRN', 'Contact Date']], 
    on=['MRN', 'Contact Date'], 
    how='inner'
)

# Keep rows where AMB Note YN is 'Y'
h_p_filtered = h_p_filtered[h_p_filtered['AMB Note  YN'] == 'N']

print(f"Number of MRNs with multiple valid dates: {len(valid_mrns)}")
print(f"Final filtered data shape: {h_p_filtered.shape}")

Number of MRNs with multiple valid dates: 224
Final filtered data shape: (9182, 7)


In [21]:
# Convert date columns to datetime
h_p_filtered['Contact Date'] = pd.to_datetime(h_p_filtered['Contact Date'])
target_list['BMT_date'] = pd.to_datetime(target_list['BMT_date'])

# Function to find matching contact date within 14 days
def find_matching_contact_date(mrn, bmt_date):
    # Get all encounter dates for this MRN
    encounter_dates = h_p_filtered[h_p_filtered['MRN'] == mrn]['Contact Date']
    
    # Find contact dates within 14 days and return the closest one
    valid_dates = []
    for encounter_date in encounter_dates:
        if 0 <=(bmt_date - encounter_date).days <= 14:
            valid_dates.append(encounter_date)
    
    if valid_dates:
        # Return the contact date closest to BMT_date
        return min(valid_dates, key=lambda x: abs((bmt_date - x).days))
    return None

# Filter target_list
# First, keep only rows where MRN exists in h_p_filtered
target_filtered = target_list[target_list['MRN'].isin(h_p_filtered['MRN'])].copy()

# Add admit_date column and filter by date criteria
target_filtered['admit_date'] = target_filtered.apply(
    lambda row: find_matching_contact_date(row['MRN'], row['BMT_date']), axis=1
)

# Keep only rows where admit_date is not None (within 14 days)
target_filtered = target_filtered[target_filtered['admit_date'].notna()]


# Retain only specified columns plus admit_date
all_columns = baseline_column + research_column + ['admit_date']
target_final = target_filtered[all_columns]

print(f"Original target_list shape: {target_list.shape}")
print(f"After MRN filtering: {target_list[target_list['MRN'].isin(h_p_list['MRN'])].shape}")
print(f"After date filtering: {target_filtered.shape}")
print(f"Final filtered dataset shape: {target_final.shape}")
print(f"Retained columns: {len(all_columns)}")

target_final.to_csv('data/target_filtered.csv', index=False)

target_final.head()

Original target_list shape: (347, 63)
After MRN filtering: (262, 63)
After date filtering: (203, 64)
Final filtered dataset shape: (203, 31)
Retained columns: 31


,MRN,BMT_date,Disease_x,age,Gn,KPS,CMV,aborh,Tx_Type,HLA,...,pressor_use_num,hypoxia_y_n,high_flow_o2_y_n,bipap_or_intubation_y_n,neurotox_y_n,toci_y_n,toci_start_date,toci_stop_date,total_dose_toci,admit_date
11,907009329,2023-12-15,MDS-CMML,63,M,90,pos,O+,MUD,7/8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-08
39,907009329,2024-12-20,AML,64,M,80,pos,O+,REL,4/8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-13
40,907163230,2024-02-22,NHL-myc fung,65,M,80,neg,A-,MUD,8/8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-16
41,980135103,2024-02-28,NHL-myc fung,58,F,80,pos,A-,MUD,8/8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-22
42,907789805,2018-02-23,AML-mutated NPM1,68,F,90,pos,A-,REL,8/8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-16


In [22]:
# Create h_p_notes by combining note text for each MRN and admission date
pt_list = pd.read_csv('data/target_filtered.csv')

# Convert admit_date to datetime for matching
pt_list['admit_date'] = pd.to_datetime(pt_list['admit_date'])

# Initialize list to store combined notes
h_p_notes_data = []

# Process each patient in the target list
for _, patient in pt_list.iterrows():
    mrn = patient['MRN']
    admit_date = patient['admit_date']
    
    # Find matching records in h_p_list for this MRN and Contact Date
    matching_records = h_p_list[
        (h_p_list['MRN'] == mrn) & 
        (pd.to_datetime(h_p_list['Contact Date']) == admit_date)
    ].copy()
    
    if not matching_records.empty:
        # Sort by Line # to ensure proper order
        matching_records = matching_records.sort_values('Line #')
        
        # Combine all note text entries into a single string
        combined_note = ' '.join(matching_records['Note Text'].fillna('').astype(str))
        
        # Add to our data list
        h_p_notes_data.append({
            'MRN': mrn,
            'admit_date': admit_date,
            'note': combined_note
        })

# Create DataFrame from the collected data
h_p_notes = pd.DataFrame(h_p_notes_data)

print(f"Created h_p_notes with {len(h_p_notes)} records")
print(f"Sample of combined notes:")
print(f"First note length: {len(h_p_notes.iloc[0]['note']) if len(h_p_notes) > 0 else 0} characters")

# Save to CSV
h_p_notes.to_csv('data/h_p_notes.csv', index=False)


Created h_p_notes with 203 records
Sample of combined notes:
First note length: 11473 characters


In [5]:
progress_1 = pd.read_excel('data/all_charts.xlsx', sheet_name='Progress Notes 1', engine='openpyxl')
progress_2 = pd.read_excel('data/all_charts.xlsx', sheet_name='Progress Notes 2', engine='openpyxl')
dc_summary = pd.read_excel('data/all_charts.xlsx', sheet_name='Discharge Summary', engine='openpyxl')
pt_list = pd.read_csv('data/target_filtered.csv')

In [25]:
progress_2[
    (progress_2['MRN'] == 907163230) & 
    (progress_2['Contact Date'] == '2024-02-23')
]


,MRN,Note ID,Contact Date,AMB Note YN,Note Type,Line #,Note Text
15956,907163230,3831063807,2024-02-23,N,Progress Notes,1,Bob is day +1 of his allo transplant for CTCL....
175723,907163230,3831735680,2024-02-23,N,Progress Notes,1,BMT SERVICE INPATIENT PROGRESS NOTE 2/23/202...
342782,907163230,3831735680,2024-02-23,N,Progress Notes,2,Allogeneic Hematopoietic Stem Cell Transplanta...
459599,907163230,3831735680,2024-02-23,N,Progress Notes,3,nutrition and strength. He also reported a sma...
588462,907163230,3831735680,2024-02-23,N,Progress Notes,4,x3. Moves all extremities. LINE: CVC Line si...
660786,907163230,3831735680,2024-02-23,N,Progress Notes,5,consulted with Dr. Brammer and the multidiscip...


In [28]:
pt_list['BMT_date'] = pd.to_datetime(pt_list['BMT_date'])
progress_1['Contact Date'] = pd.to_datetime(progress_1['Contact Date'])
progress_2['Contact Date'] = pd.to_datetime(progress_2['Contact Date'])

progress_notes_data = []

for _, patient in pt_list.iterrows():
    mrn = patient['MRN']
    bmt_date = patient['BMT_date']
    
    # Look for notes from bmt_date to bmt_date + 5
    for day_offset in range(0, 6):  # Days 0, 1, 2, 3, 4, 5
        target_date = bmt_date + pd.Timedelta(days=day_offset)
        
        # Check progress_1 first
        progress_1_matches = progress_1[
            (progress_1['MRN'] == mrn) & 
            (progress_1['Contact Date'] == target_date) &
            (progress_1['Note Text'].str.contains('BMT SERVICE INPATIENT', na=False))
        ]
        
        matching_note_id = None
        source_sheet = None
        source_df = None
        
        if not progress_1_matches.empty:
            matching_note_id = progress_1_matches['Note ID'].values[0]
            source_sheet = 'Progress Notes 1'
            source_df = progress_1
        else:
            # If no records found in progress_1, check progress_2
            progress_2_matches = progress_2[
                (progress_2['MRN'] == mrn) & 
                (progress_2['Contact Date'] == target_date) &
                (progress_2['Note Text'].str.contains('BMT SERVICE INPATIENT', na=False))
            ]
            
            if not progress_2_matches.empty:
                matching_note_id = progress_2_matches['Note ID'].values[0]
                source_sheet = 'Progress Notes 2'
                source_df = progress_2
        
        # If records found, combine the notes
        if matching_note_id is not None and source_df is not None:
            # Get ALL lines for this Note ID (spanning multiple Line # values)
            matching_records = source_df[source_df['Note ID'] == matching_note_id].sort_values('Line #')
            
            # Combine all note text entries into a single string
            combined_note = ' '.join(matching_records['Note Text'].fillna('').astype(str))
            
            # Add to our data list
            progress_notes_data.append({
                'MRN': mrn,
                'bmt_date': bmt_date,
                'note_date': day_offset,
                'note': combined_note,
                'source_sheet': source_sheet
            })

# Create DataFrame from the collected data
progress_notes = pd.DataFrame(progress_notes_data)

print(f"Created progress_notes with {len(progress_notes)} records")
print(f"Days covered: {sorted(progress_notes['note_date'].unique()) if len(progress_notes) > 0 else []}")

if len(progress_notes) > 0:
    print(f"\nRecord count by day:")
    print(progress_notes['note_date'].value_counts().sort_index())

# Save to CSV
progress_notes.to_csv('data/progress_notes_new.csv', index=False)

Created progress_notes with 1119 records
Days covered: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]

Record count by day:
note_date
0    187
1    186
2    185
3    184
4    188
5    189
Name: count, dtype: int64


In [29]:
# keep only those mrn in h_p_notes in progress_notes
import pandas as pd

h_p_notes = pd.read_csv('data/h_p_notes.csv')
progress_notes = pd.read_csv('data/progress_notes.csv')

h_p_notes['MRN'] = h_p_notes['MRN'].astype(str)
progress_notes['MRN'] = progress_notes['MRN'].astype(str)

# keep only those mrn in h_p_notes in progress_notes
progress_notes = progress_notes[progress_notes['MRN'].isin(h_p_notes['MRN'])]

# remove character Â°, Â , Â from note column
progress_notes['note'] = progress_notes['note'].str.replace('Â°', '')
progress_notes['note'] = progress_notes['note'].str.replace('Â ', '')
progress_notes['note'] = progress_notes['note'].str.replace('Â', '')

progress_notes.to_csv('data/progress_notes.csv', index=False)

In [6]:
pt_list['admit_date'] = pd.to_datetime(pt_list['admit_date'])
dc_summary['Contact Date'] = pd.to_datetime(dc_summary['Contact Date'])

dc_summary_data = []

# Process each patient in the target list
for _, patient in pt_list.iterrows():
    mrn = patient['MRN']
    admit_date = patient['admit_date']
    
    # Get all discharge summary dates for this MRN that are on or after admission date
    mrn_dc_records = dc_summary[dc_summary['MRN'] == mrn].copy()
    
    if not mrn_dc_records.empty:
        # Find contact dates that are >= admit_date
        valid_dc_dates = mrn_dc_records[mrn_dc_records['Contact Date'] > admit_date]['Contact Date']
        
        if not valid_dc_dates.empty:
            # Find the closest date after admission (minimum difference)
            discharge_date = min(valid_dc_dates, key=lambda x: (x - admit_date).days)
            
            # Get records for this specific discharge date
            matching_records = dc_summary[
                (dc_summary['MRN'] == mrn) & 
                (dc_summary['Contact Date'] == discharge_date)
            ].copy()
            
            if not matching_records.empty:
                # Sort by Line # to ensure proper order
                matching_records = matching_records.sort_values('Line #')
                
                # Combine all note text entries into a single string
                combined_note = ' '.join(matching_records['Note Text'].fillna('').astype(str))
                
                # Calculate days between admission and discharge
                days_to_discharge = (discharge_date - admit_date).days
                
                # Add to our data list
                dc_summary_data.append({
                    'MRN': mrn,
                    'admit_date': admit_date,
                    'discharge_date': discharge_date,
                    'days_to_discharge': days_to_discharge,
                    'note': combined_note
                })

# Create DataFrame from the collected data
dc_summary_data = pd.DataFrame(dc_summary_data)

print(f"Created dc_summary_data with {len(dc_summary_data)} records")

if len(dc_summary_data) > 0:
    print(f"Average days to discharge: {dc_summary_data['days_to_discharge'].mean():.1f}")
    print(f"Min days to discharge: {dc_summary_data['days_to_discharge'].min()}")
    print(f"Max days to discharge: {dc_summary_data['days_to_discharge'].max()}")
    print(f"Sample of combined notes:")
    print(f"First note length: {len(dc_summary_data.iloc[0]['note'])} characters")
else:
    print("No discharge summary records found")

# Save to CSV
dc_summary_data.to_csv('data/dc_summary_notes.csv', index=False)

Created dc_summary_data with 202 records
Average days to discharge: 30.3
Min days to discharge: 20
Max days to discharge: 112
Sample of combined notes:
First note length: 48125 characters


In [ ]:
import pandas as pd

h_p_notes = pd.read_csv('data/h_p_notes.csv')
dc_summary_data = pd.read_csv('data/dc_summary_notes.csv')
target_filtered = pd.read_csv('data/target_filtered.csv')

# remove duplicate mrn from h_p_notes
h_p_notes = h_p_notes.drop_duplicates(subset=['MRN'], keep=False)

# remove duplicate mrn from dc_summary_data
dc_summary_data = dc_summary_data.drop_duplicates(subset=['MRN'], keep=False)

# remove duplicate mrn from target_final
target_filtered = target_filtered.drop_duplicates(subset=['MRN'], keep=False)

h_p_notes.to_csv('data/h_p_notes.csv', index=False)
dc_summary_data.to_csv('data/dc_summary_notes.csv', index=False)
target_filtered.to_csv('data/target_filtered.csv', index=False)

In [3]:
# for h_p_notes, remove column 'Note' containing 'Pre Procedure H&P'
import pandas as pd

h_p_notes = pd.read_csv('data/h_p_notes.csv')

h_p_notes = h_p_notes[~h_p_notes['note'].str.contains('Pre Procedure H&P')]

h_p_notes.to_csv('data/h_p_notes.csv', index=False)

In [2]:
# Filter toci_ptcy_crs.csv to keep only rows where MRN exists in h_p_list
import pandas as pd

target_list = pd.read_csv('data/toci_ptcy_crs.csv')
h_p_list = pd.read_csv('data/h_p_notes.csv')

target_list = target_list[target_list['MRN'].isin(h_p_list['MRN'])]

# Save the filtered data to a new CSV file
target_list.to_csv('data/toci_ptcy_crs_filtered.csv', index=False)

In [4]:
# simplify disease name in target filtered
import pandas as pd

target_filtered = pd.read_csv('data/target_filtered.csv')

target_filtered['Disease_x_full_name'] = target_filtered['Disease_x']

def process_disease_name(disease):
    if disease == 'MDS-MPN':
        return disease
    elif '-' in disease:
        return disease.split('-')[0]
    else:
        return disease

target_filtered['Disease_x'] = target_filtered['Disease_x'].apply(process_disease_name)

target_filtered.to_csv('data/target_filtered.csv', index=False)

In [ ]:
# calculate total dose numbers of tocilizumab from total mg of toci

import pandas as pd
import numpy as np

toci_data = pd.read_excel('data/toci_ptcy_crs.xlsx', engine='openpyxl')

toci_data['total_dose_toci'] = (
    toci_data['total_dose_toci']
    .astype(str)
    .str.replace('mg', '', case=False, regex=False)
    .str.strip()
)
toci_data['total_dose_toci'] = pd.to_numeric(toci_data['total_dose_toci'], errors='coerce')

conditions = [
    toci_data['total_dose_toci'] > 900,
    (toci_data['total_dose_toci'] > 0) & (toci_data['total_dose_toci'] <= 900),
    toci_data['total_dose_toci'].isna()
]

# Corresponding values
choices = [2, 1, 0]

# Create the new column
toci_data['total_dose_toci'] = np.select(conditions, choices, default=0)
toci_data.to_csv('data/toci_ptcy_crs.csv', index=False)